In [3]:
# ==== 設定 ====
STORAGE = "sqlite:///lstmgru_tuning.db"
STUDY   = "imu_tuning"

import optuna, pandas as pd, numpy as np, matplotlib.pyplot as plt
from optuna.trial import TrialState

study = optuna.load_study(study_name=STUDY, storage=STORAGE)

# 1) user_attr から fold_scores(list[float]) を収集
rows = []
for t in study.get_trials(states=(TrialState.COMPLETE,)):
    fs = t.user_attrs.get("fold_scores", None)
    if isinstance(fs, (list, tuple)) and len(fs) > 0:
        for i, s in enumerate(fs, start=1):  # i=fold番号(1始まり)
            rows.append({"trial": t.number, "fold": i, "score": float(s), "objective": t.value})

df = pd.DataFrame(rows)
assert not df.empty, "fold_scores が見つかりません。trial.set_user_attr('fold_scores', list) を確認してください。"

# 2) ピボット（trial×fold）
pv = df.pivot_table(index="trial", columns="fold", values="score", aggfunc="mean")
pv["mean"] = pv.mean(axis=1)

# 3) 上位を確認
top = pv.sort_values("mean", ascending=False)
print("Top trials by mean score:\n", top.head(10))

# 4) ヒートマップ（上位20試行）
top_k = min(20, len(top))
hm = top.iloc[:top_k, :-1]  # 平均列を除外
plt.figure(figsize=(10, max(4, 0.35*top_k)))
plt.imshow(hm, aspect="auto")
plt.title(f"Fold scores per trial (top {top_k})")
plt.xlabel("fold")
plt.ylabel("trial")
plt.colorbar(label="score")
plt.yticks(range(top_k), hm.index.to_list())
plt.show()

# 5) 上位5試行の平均スコア棒グラフ
plt.figure(figsize=(6,4))
top["mean"].head(min(5, len(top))).plot(kind="bar")
plt.ylabel("mean score")
plt.title("Top trials (mean over folds)")
plt.tight_layout()
plt.show()

# 6) ベスト試行の fold別スコアを個別に可視化（任意）
best_trial = int(top.index[0])
plt.figure(figsize=(6,3))
pv.loc[best_trial].drop("mean").plot(kind="bar")
plt.ylabel("score")
plt.title(f"Best trial #{best_trial} - per-fold scores")
plt.tight_layout()
plt.show()

# （必要ならCSV出力）
# pv.to_csv("fold_scores_pivot.csv")


KeyError: 'Record does not exist.'

In [4]:
import optuna
# 1) そのstorageにある全study名を確認
print([s.study_name for s in optuna.study.get_all_study_summaries(storage=STORAGE)])

# 2) SQLite を使っている場合、URLが正しいか（WSLなら /mnt/c の絶対パス推奨）
# 例: STORAGE = "sqlite:////mnt/c/Users/tkdgk/optuna/optuna.db"  # 先頭スラッシュ4つ


[]


In [5]:
import os
os.listdir()

['config',
 'datamodule.py',
 'f_output',
 'lightning_logs',
 'lstmgru_tuning.db',
 'outputs',
 'result.ipynb',
 'result.py',
 'train.py',
 'train.sh',
 'tuning.py',
 'utils.py',
 '__pycache__']